In [27]:
import os
import re
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from dataprep.clean import clean_country, validate_country
import warnings
warnings.filterwarnings("ignore")

In [28]:
DATA = 'data'

In [29]:
production_df = pd.read_csv(os.path.join(DATA, 'Production_Crops_Livestock_E_All_Data.csv'), encoding='ISO-8859-1')
production_df['Code'] = production_df['Area'].str[:3]  # create the 3-letter country code we need
production_df['Code'] = production_df['Code'].str.upper()
code = production_df.pop('Code')  # pop the code column
production_df.insert(2, 'Code', code)  # move it to the position we want
production_df.head()
# deal with the warnings

,Area Code,Area,Code,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,...,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F,Y2020,Y2020F
0,2,Afghanistan,AFG,221,"Almonds, with shell",5312,Area harvested,ha,NaN,NaN,...,19481.0,NaN,19793.0,NaN,20053.0,NaN,29203.0,NaN,22134.0,NaN
1,2,Afghanistan,AFG,221,"Almonds, with shell",5419,Yield,hg/ha,NaN,NaN,...,16859.0,Fc,13788.0,Fc,17161.0,Fc,13083.0,Fc,17759.0,Fc
2,2,Afghanistan,AFG,221,"Almonds, with shell",5510,Production,tonnes,NaN,NaN,...,32843.0,NaN,27291.0,NaN,34413.0,NaN,38205.0,NaN,39307.0,NaN
3,2,Afghanistan,AFG,711,"Anise, badian, fennel, coriander",5312,Area harvested,ha,NaN,M,...,24500.0,Im,26500.0,Im,25333.0,Im,25444.0,Im,25759.0,Im
4,2,Afghanistan,AFG,711,"Anise, badian, fennel, coriander",5419,Yield,hg/ha,NaN,NaN,...,7075.0,Fc,7149.0,Fc,7142.0,Fc,7123.0,Fc,7138.0,Fc


# Why?
I became interested in the drivers of the price per mt of RBD Palm Olein FOB Malaysia, as i worked for an oil bottler buying approx 2ML per month. As the price increased from USD515 p/mt on 7th May 2020 to USD1064 on 1st March 2021, I found the explanations being given to customers unsatisfactory.

From a purchasing and operations management perspective, it also became a challenge. It makes sense to buy forward in a climbing or stable market, foolish to buy forward in a declining market, and dangerous in a volatile market. There were many opinions but no reliable solutions.

I had questions:
- What had driven this rapid increase?
- If this was a bubble, when would it pop?
- If this was a structural change, what were the changes?
- In an uncertain business environment, how can buyers use publicly available data to make better purchasing decisions?


In [30]:
price = pd.read_csv(os.path.join(DATA, 'palm oil prices 021020 - 290422.csv'))
price.columns = ['date', 'price']
price['price'] = price['price'].ffill()
fig = px.line(price, x='date', y='price')
fig.update_layout(
    title_text="<b>Global Palm Oil Price<b>",
    title_font_size=40,
    legend_font_size=20,
    width=800,
    height=400
)  # Add figure title
fig.show()

# why am i missing vert & horizontal lines?how to reduce the number of years shown on the x axis so its less noisy and more readable

So i began researching. I started with the macro aspects of not only the palm oil market, but also, because they are substitutes, the vegetable oil market in general.

# Set-up

In [31]:
production_elements = [
    'Area harvested',
    'Yield',
    'Production',
    'Stocks',
    'Laying',
    'Producing Animals/Slaughtered',
    'Yield/Carcass Weight',
    'Milk Animals',
    'Prod Popultn'
]

In [32]:
# area codes >1000 and in this list are regions and will cause double count unless removed
area_code_list = [
    261,
    265,
    266,
    268,
    269
]

In [33]:
oil_crops = [
    'Oil, castor beans',
    'Oil, citronella',
    'Oil, coconut (copra)',
    'Oil, cottonseed',
    'Oil, essential nes',
    'Oil, groundnut',
    'Oil, linseed',
    'Oil, maize',
    'Oil, olive residues',
    'Oil, olive, virgin',
    'Oil, palm',
    'Oil, palm kernel',
    'Oil, rapeseed',
    'Oil, sesame',
    'Oil, soybean',
    'Oil, sunflower'
]

In [34]:
# Wrangle production data 🗸
production_df = pd.read_csv(os.path.join(DATA, 'Production_Crops_Livestock_E_All_Data.csv'), encoding='ISO-8859-1')
# (77523, 127, 76.86mb)
production_df = production_df.groupby(['Item']).sum()
production_df = production_df.drop(['Item Code', 'Area Code', 'Element Code'], axis=1)
production_df.columns = production_df.columns.str.replace('Y', '')
production_df = production_df.loc[:,~production_df.columns.str.endswith('F')]
production_df = production_df[production_df.index.isin(oil_crops)].transpose()
production_df.index.name = "Year"
production_df = production_df.reset_index(drop=False)
production_df = production_df[production_df.Year != '2020'].copy()
production_df.head()
# print(production_df.shape)

Item,Year,"Oil, coconut (copra)","Oil, cottonseed","Oil, groundnut","Oil, linseed","Oil, maize","Oil, olive, virgin","Oil, palm","Oil, palm kernel","Oil, rapeseed","Oil, sesame","Oil, soybean","Oil, sunflower"
0,1961,7515823.0,9500322.0,12746566.0,3688186.0,1470757.0,6626752.0,7161016.0,2357251.0,5403878.0,1985736.0,12792646.0,8150917.0
1,1962,8925274.0,9911752.0,13289585.0,4177110.0,1545697.0,4553847.0,7171496.0,2339709.0,5761541.0,2379515.0,14014734.0,9572099.0
2,1963,8668362.0,10803476.0,14653169.0,4001376.0,1589790.0,8871607.0,7429088.0,2165697.0,5568586.0,2380065.0,14958936.0,10076964.0
3,1964,8367046.0,11580974.0,15166158.0,4040149.0,1674965.0,4506758.0,7548695.0,2374090.0,5377562.0,2398931.0,14990326.0,9903156.0
4,1965,8649448.0,12268292.0,13742532.0,4507195.0,1837119.0,6092509.0,7458087.0,2452860.0,7559540.0,2418330.0,16236072.0,12540614.0


# How has demand for edible oil changed over time? How much has the market grown in the last 60 years?

In [35]:
# 🗸
veg_oil_prodn_fig = px.area(production_df, x='Year', y=production_df.columns[1:])
veg_oil_prodn_fig.update_traces(textfont_size=16, hovertemplate=None)
veg_oil_prodn_fig.update_layout(hovermode="x")

veg_oil_prodn_fig.update_layout(
    title_text="<b>Global Vegetable Oil Production<b>",
    title_font_size=40,
    legend_font_size=20,
    width=1000,
    height=700
)  # Add figure title

veg_oil_prodn_fig.update_xaxes(
    title_text="</b>Year</b>",
    title_font=dict(size=30, family='Verdana', color='white'),
    tickfont=dict(family='Calibri', color='white', size=25)
)  # format x-axis

veg_oil_prodn_fig.update_yaxes(
    title_text="<b>Palm Oil Fruit (mt)</b>",
    title_font=dict(size=30, family='Verdana', color='white'),
    tickfont=dict(family='Calibri', color='white', size=25)
)  # Format y-axes

veg_oil_prodn_fig.show()

# Conclusion 1:
- the oil market has grown by 10x in the 60 years since 1961
- palm oil has overtaken traditional crops like soybean and rapeseed as the largest contributor to global edible oil production

# Palm Oil Production

In [36]:
palm_oil_production = production_df[["Year", "Oil, palm"]]  # new df
palm_oil_prodn_fig = px.line(palm_oil_production, x="Year", y="Oil, palm")

palm_oil_prodn_fig.update_layout(
    title_text="<b>Global Oil Palm Production<b>",
    title_font_size=40,
    legend_font_size=20,
    width=1400,
    height=1000
)

palm_oil_prodn_fig.update_xaxes(
    title_text="Year",
    title_font=dict(size=30, family='Verdana', color='white'),
    tickfont=dict(family='Calibri', color='white', size=25)
)

palm_oil_prodn_fig.update_yaxes(
    title_text="<b>Palm Oil production (mt)</b>",
    title_font=dict(size=30, family='Verdana', color='white'),
    tickfont=dict(family='Calibri', color='white', size=25)
)

palm_oil_prodn_fig.show()

# Conclusion 2:
- palm oil production capacity has continued to grow very rapidly to keep up with demand.
- in 1961 total global production of palm oil was 7.6 million tons
- by 2019 that had increaseed to 304.3 million tons, an increase of 40x

# How has production of palm oil changed by location over time?
### Geo Plot of production

In [37]:
production_df = pd.read_csv(os.path.join(DATA, 'Production_Crops_Livestock_E_All_Data.csv'), encoding='ISO-8859-1')
production_df = production_df.drop(['Area Code', 'Item Code', 'Element Code', 'Element', 'Unit'], axis=1)
production_df.columns = production_df.columns.str.replace('Y', '')
production_df = production_df.loc[:,~production_df.columns.str.endswith('F')]
production_df = production_df[production_df['Item'].isin(oil_crops)].reset_index(drop=True)
production_df['Code'] = production_df['Area'].str[:3]
production_df['Code'] = production_df['Code'].str.upper()
code = production_df.pop('Code')  # pop the code column
production_df.insert(1, 'Code', code)  # move it to the position we want
prodction_df = production_df[~production_df['Code'].isnull()].copy()
temp_df = production_df.melt(id_vars=production_df.columns[:3], value_vars=production_df.columns[3:])
temp_df.rename(columns={'variable': 'Year'}, inplace=True)
new_production_df = temp_df.pivot_table(index=['Area', 'Code', 'Year'], columns=['Item'], values='value', aggfunc=sum).reset_index()
production_df = new_production_df.rename(columns={'Area': 'Entity'})  # this isn't strictly necessary. just to match wilson's work.
# need a pivot to get 'Year' into a column and 'oil type' as the column header from [3:]
production_df = production_df[production_df.Year != '2020'].copy()

In [38]:
oil_type = 'Oil, cottonseed'
geo_fig = px.choropleth(
    production_df,
    locations='Code',
    color=oil_type,
    color_continuous_scale=px.colors.diverging.PiYG,
    locationmode='ISO-3',
    animation_frame='Year',
    projection='natural earth'
)
geo_fig.update_layout(
    title_text=f'{oil_type}',
)

geo_fig.show()

## I would like 3 choropleth maps with animation: 1 for production, 1 for imports, 1 for exports.

In [ ]:
pattern = r'(?<=Oil, ).+?(?= - \d)'
cols = [re.search(pattern, c, re.RegexFlag.IGNORECASE)[0] for c in veg_oil_yearly_production]
cols = [re.sub(' ', '_', c) for c in cols]
cols = [re.sub('\W', '', c) for c in cols]

veg_oil_yearly_production.columns = cols
veg_oil_yearly_production.reset_index(inplace=True)
veg_oil_yearly_production.info()

cols_to_rename = production_df.columns[3:]

cols = [re.search(pattern, c, re.RegexFlag.IGNORECASE)[0] for c in cols_to_rename]
cols = [re.sub(' ', '_', c) for c in cols]
cols = [re.sub('\W', '', c) for c in cols]

veg_oil_by_country.columns = veg_oil_by_country.columns.tolist()[:3] + cols
veg_oil_by_country

In [39]:
# the first choropleth map, not animated
veg_oil_type = 'palm'
fig = go.Figure(
    data=go.Choropleth(
        locations=veg_oil_by_country['Code'],
        z=veg_oil_by_country[veg_oil_type],
        locationmode='ISO-3',  #  "ISO-3" | "USA-states" | "country names"
        colorscale='Viridis',
        colorbar_title=veg_oil_type
    )
)
fig.update_layout(title_text = f'{veg_oil_type} Vegetable Oil')
fig.show()

NameError: name 'veg_oil_by_country' is not defined

## I would like 3 choropleth maps with animation: 1 for production, 1 for imports, 1 for exports.

# Conclusion 3:
- the majority of palm oil is produced in Malaysia & Indonesia
- Earlier, Malaysia was the leading producer, but more recently Indonesia has become the dominant producer
- There are limits to further increases for both nations as the primary driver to production capacity is suitable land.
- Both countries have finite supply of suitable land

# Trade



In [40]:
trade_df = pd.read_csv(os.path.join(DATA, 'Trade_CropsLivestock_E_All_Data.csv'), encoding='ISO-8859-1')
trade_df.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F,Y2020,Y2020F
0,2,Afghanistan,862,Alfalfa meal and pellets,5910,Export Quantity,tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,49.0,*,21.0,*
1,2,Afghanistan,862,Alfalfa meal and pellets,5922,Export Value,1000 US$,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.0,*,8.0,*
2,2,Afghanistan,231,Almonds shelled,5610,Import Quantity,tonnes,NaN,NaN,NaN,...,845.0,R,343.0,NaN,597.0,R,3520.0,R,2860.0,R
3,2,Afghanistan,231,Almonds shelled,5622,Import Value,1000 US$,NaN,NaN,NaN,...,4988.0,R,1021.0,NaN,1242.0,R,25540.0,R,17865.0,R
4,2,Afghanistan,231,Almonds shelled,5910,Export Quantity,tonnes,0.0,NaN,0.0,...,1691.0,R,3335.0,R,3339.0,R,6848.0,R,4123.0,R


# Export: Which countries export palm oil?
i expect exports to closely mirror production

In [41]:
exports_df = trade_df[trade_df['Element'].str.contains('Export Quantity') == True]
exports_df = exports_df[exports_df['Item'].isin(oil_crops)==True].drop(columns=['Area Code', 'Item Code', 'Element Code'])
exports_df = exports_df.groupby(by=['Item']).sum()
exports_df.columns = exports_df.columns.str.replace('Y', '')
print(exports_df)

                           1961       1962       1963       1964       1965  \
Item                                                                          
Oil, castor beans      541574.0   488331.0   565880.0   625659.0   679446.0   
Oil, citronella          3161.0     3907.0     5102.0     5328.0     4338.0   
Oil, coconut (copra)  1619467.0  1957845.0  2054934.0  2226978.0  2066083.0   
Oil, cottonseed       1004715.0   953943.0  1025731.0  1426877.0  1543712.0   
Oil, essential nes      33993.0    39425.0    39240.0    40373.0    42933.0   
Oil, groundnut        1638205.0  2003106.0  2037873.0  2233976.0  2312798.0   
Oil, linseed          1117935.0  1219336.0  1096184.0  1026825.0  1227316.0   
Oil, maize              36970.0    43655.0    27070.0    33405.0    38665.0   
Oil, olive residues     12665.0    47020.0    28245.0    16185.0    31205.0   
Oil, olive, virgin    1008692.0  1054693.0   770927.0  1023002.0   621821.0   
Oil, palm             3117913.0  2901591.0  2952325.

In [42]:
# choropleth map
geo_fig = px.choropleth(
    production_df,  # the dataframe
    locations='Item',
    color=production_df['Oil, palm'],
    color_continuous_scale=px.colors.diverging.PiYG,
    locationmode='ISO-3',
    animation_frame='Year',
    projection='natural earth'
)
geo_fig.update_layout(title_text = f'Palm Oil Exports',)

geo_fig.show()

ValueError: Value of 'locations' is not the name of a column in 'data_frame'. Expected one of ['Entity', 'Code', 'Year', 'Oil, coconut (copra)', 'Oil, cottonseed', 'Oil, groundnut', 'Oil, linseed', 'Oil, maize', 'Oil, olive, virgin', 'Oil, palm', 'Oil, palm kernel', 'Oil, rapeseed', 'Oil, sesame', 'Oil, soybean', 'Oil, sunflower'] but received: Item

# Help! ⬇
### Imports: Which countries import Palm Oil?

In [43]:
imports_df = trade_df[trade_df['Element'].str.contains('Import Quantity') == True]
imports_df = imports_df.drop(imports_df[imports_df['Area Code'] > 1000].index)
imports_df = imports_df.drop(imports_df[imports_df['Area Code'].isin(area_code_list)].index)
imports_df = imports_df[imports_df['Item'].isin(oil_crops)==True].drop(columns=[ 'Item Code', 'Element Code'])
imports_df.columns = imports_df.columns.str.replace('Y', '')
imports_df = imports_df.loc[:,~imports_df.columns.str.endswith('F')]
# validate the country and generate unicode-3 for each country
imports_df['country_val'] = validate_country(imports_df["Area"])
# remove the few countries that don't pass validation
imports_df = imports_df.loc[imports_df['country_val'] != False]
# generate the utf-3 code for each country
imports_df = clean_country(df=imports_df, column="Area", output_format='alpha-3')
imports_df

  0%|          | 0/8 [00:00<?, ?it/s]

Country Cleaning Report:
	2768 values cleaned (100.0%)
Result contains 2768 (100.0%) values in the correct format and 0 null values (0.0%)


,Area Code,Area,Item,Element,Unit,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,country_val,Area_clean
811,2,Afghanistan,"Oil, castor beans",Import Quantity,tonnes,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,1.0,NaN,2.0,2.0,2.0,True,AFG
815,2,Afghanistan,"Oil, coconut (copra)",Import Quantity,tonnes,NaN,NaN,NaN,NaN,NaN,...,NaN,35.0,117.0,25.0,NaN,1.0,1316.0,1.0,True,AFG
819,2,Afghanistan,"Oil, cottonseed",Import Quantity,tonnes,NaN,NaN,NaN,NaN,NaN,...,NaN,19.0,NaN,NaN,NaN,NaN,30.0,10660.0,True,AFG
821,2,Afghanistan,"Oil, essential nes",Import Quantity,tonnes,NaN,NaN,NaN,NaN,NaN,...,NaN,149.0,0.0,143.0,NaN,32.0,5.0,5.0,True,AFG
825,2,Afghanistan,"Oil, groundnut",Import Quantity,tonnes,0.0,0.0,0.0,10.0,5.0,...,0.0,0.0,NaN,1.0,17178.0,NaN,NaN,NaN,True,AFG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282260,181,Zimbabwe,"Oil, palm kernel",Import Quantity,tonnes,0.0,0.0,0.0,0.0,0.0,...,193.0,114.0,60.0,0.0,34.0,297.0,347.0,249.0,True,ZWE
282264,181,Zimbabwe,"Oil, rapeseed",Import Quantity,tonnes,0.0,0.0,0.0,0.0,0.0,...,0.0,22.0,10.0,126.0,215.0,70.0,65.0,0.0,True,ZWE
282270,181,Zimbabwe,"Oil, sesame",Import Quantity,tonnes,NaN,NaN,NaN,NaN,NaN,...,5.0,0.0,0.0,1.0,NaN,0.0,0.0,2.0,True,ZWE
282274,181,Zimbabwe,"Oil, soybean",Import Quantity,tonnes,0.0,0.0,0.0,0.0,0.0,...,27331.0,47758.0,104655.0,121158.0,96173.0,121304.0,74781.0,124126.0,True,ZWE


In [44]:
# I'd like a choropleth map with animation for imports
imports_geo_fig = px.choropleth(
    imports_df,
    locations='Area',
    color=oil_crops,
    color_continuous_scale=px.colors.sequential.Oranges,
    locationmode='ISO-3',
    projection='natural earth'
)

imports_geo_fig.update_layout(
    title_text="<b>Global Palm Oil Imports<b>",
    title_font_size=40,
    legend_font_size=20,
    width=1100,
    height=750
)

imports_geo_fig.show()

ValueError: All arguments should have the same length. The length of argument `color` is 16, whereas the length of  previously-processed arguments ['Area'] is 2768

# Conclusion 4:
the two dominant importers are India and China. This is not surprising: growth in imports over the time period driven by growing populations and per capita GDP.